In [1]:
from jupyter_dash import JupyterDash 
import dash_core_components as dcc 
import dash_html_components as html 
import plotly.express as px 
import pandas as pd


from dash.dependencies import Input, Output

In [2]:
def read_posdata(path):
    df = pd.read_table(path, encoding='utf-16', header=[1,2,3,4,5])
    df = df.melt(id_vars=[df.columns[0]])
    df.columns = ['date', 'area', 'store', 'huge', 'mid', 'type', 'value']
    return df

In [3]:
df = read_posdata('data/pos_data_0706.csv')

In [4]:
dff = df[df['area'] == '00_全国']
dfff = dff[dff['store'] == '01_スーパーマーケット']
dfff.huge.unique()

array(['00_大分類計', '10_食品', '20_飲料', '30_雑貨', '40_化粧品', '50_ヘルスケア',
       '60_その他'], dtype=object)

In [5]:
df.columns

Index(['date', 'area', 'store', 'huge', 'mid', 'type', 'value'], dtype='object')

In [6]:
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

app.layout = html.Div([
    
    dcc.RadioItems(
        id = 'my_radio',
        options = [{"label": t, "value": t} for t in df.type.unique()],
        value = '販売金額指数'
    ),
    
    html.Div(id='my_body',
        

    )
    
])

@app.callback(Output('my_body', 'children'), [Input('my_radio', 'value')])
def update_body(selected_value):
    if selected_value == '前年比（％）':
        
        return html.Div()
    
    else:
        index_body = html.Div([
            
            dcc.Dropdown(
                id='drop_one',
                options=[{'label': a, 'value': a} for a in df.area.unique()],
                value=df.area.unique()[0]
                    ),
            dcc.Dropdown(id='drop_two'),
            dcc.Dropdown(id='drop_thr'),
            dcc.Dropdown(id='drop_four'),
            dcc.Graph(id='my_graph')
            
        ])
        
        
        return index_body
    
@app.callback([Output('drop_two', 'options'), 
               Output('drop_two', 'value')],
               [Input('drop_one', 'value')])
def update_drop_two(selected_value):
    df1 = df[df['area'] == selected_value]
    
    return [{'label': st, 'value': st} for st in df1['store'].unique()], df1['store'].unique()[0]

@app.callback([Output('drop_thr', 'options'),
              Output('drop_thr', 'value')],
              [Input('drop_two', 'value')]
             )
def update_drop_three(selected_value):
    df1 = df[df['store'] == selected_value]
    
    return [{'label': h, 'value': h} for h in df1['huge'].unique()], df1['huge'].unique()[0]

@app.callback([Output('drop_four', 'options'),
              Output('drop_four', 'value'),
              Output('drop_four', 'multi')
              ],
             [Input('drop_thr', 'value')]
             )
def update_drop_four(selected_value):
    df1 = df[df['huge'] == selected_value]
    
    return [{'label': h, 'value': h} for h in df1['mid'].unique()], [df1['mid'].unique()[0]], True

@app.callback(Output('my_graph', 'figure'), 
              [Input('drop_one', 'value'),
               Input('drop_two', 'value'),
               Input('drop_thr', 'value'),
               Input('drop_four', 'value'),
              Input('my_radio', 'value')
              ])
def update_graph(selected_one, selected_two, selected_three, selected_four, radio_value):
    df1 = df[df['area'] == selected_one]
    df2 = df1[df1['store'] == selected_two]
    df3 = df2[df2['huge'] == selected_three]
    df4 = df3[df3['type'] == radio_value]
    df5 = df4[df4['mid'].isin(selected_four)]
    
    return px.line(df5, x='date', y='value', color='mid')


In [7]:
app.run_server(mode='jupyterlab')